## 異なるタスクのモデル比較

このノートブックでは、Mistral-7Bと並行して別のモデルであるFlan-T5-largeを使用し、その挙動を観察します。

Flan-T5-Largeは確かに小型で、GPUなしで動作し、4GBのRAMしか使用しませんが、タスクをこなすことができるのでしょうか？

### 要件とインポート

ラボの指示に従って適切なワークベンチイメージを選択して起動していれば、必要なライブラリはすでに揃っているはずです。そうでない場合は、次のセルの最初の行のコメントを外して、正しいパッケージをすべてインストールしてください。

In [16]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # 正しいワークベンチイメージを選択していない場合のみコメントを外してください

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI, HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Langchainパイプライン

これから2つの異なるLLMエンドポイントと、2つの異なるLangchainパイプラインを定義します。

In [17]:
# LLM推論サーバーURL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:8000"

# LLMの定義
llm = VLLMOpenAI(           # vLLM OpenAI互換APIクライアントを使用していますが、モデルはOpenShift上で動作しています。
    openai_api_key="EMPTY",   # そのため、OpenAIのキーは不要です。
    openai_api_base= f"{inference_server_url}/v1",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [18]:
# Flan-T5-Small LLM推論サーバーURL
inference_server_url_flan_t5 = "http://llm-flant5.ic-shared-llm.svc.cluster.local:3000/"

# LLMの定義
llm_flant5 = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url_flan_t5,
    max_new_tokens=96,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

両方のモデルに対して**テンプレート**は同じになります。

In [19]:
template="""<s>[INST]
You are a helpful, respectful and honest assistant.
Always assist with care, respect, and truth. Respond with utmost utility yet securely.
Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.
I will give you a text, then ask a question about it. Give a precise and as concise as possible answer to this question.

### TEXT:
{text}

### QUESTION:
{query}

### ANSWER:
[/INST]
"""

これで、2つの**会話**オブジェクトを作成し、2つのモデルにクエリを投げる準備が整いました。

In [20]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )
conversation_flant5 = LLMChain(llm=llm_flant5,
                        prompt=PROMPT,
                        verbose=False
                        )

モデルにクエリを投げる準備が整いました！

この例では、1つの主張をクエリして、どのような結果が得られるかを見てみます。もちろん、他の主張で試してみても構いません。

In [23]:
filename = 'claims/claim1.json'

# Opening JSON file
claims = {}
with open(filename, 'r') as file:
    data = json.load(file)
claims[filename] = data

# Analyze the claim
print(f"***************************")
print(f"* Claim: {filename}")
print(f"***************************")
print("Original content:")
print("-----------------")
print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
print('Analysis with Mistral-7B:')
print("--------")
text_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
sentiment_query = "What is the sentiment of the person sending this claim?"
location_query = "Where does the event the claim is related to happen?"
time_query = "When does the event the claim is related to happen? If possible, specify the date and the time."
print(f"- Sentiment: ")
conversation.predict(text=text_input, query=sentiment_query);
print("\n- Location: ")
conversation.predict(text=text_input, query=location_query);
print("\n- Time: ")
conversation.predict(text=text_input, query=time_query);
print("\n\n                          ----====----\n")
print('Analysis with Flan-T5-Large:')
print("--------")
text_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
sentiment_query = "What is the sentiment of the person sending this claim?"
location_query = "Where does the event the claim is related to happen?"
time_query = "When does the event the claim is related to happen? If possible, specify the date and the time."
print(f"- Sentiment: ")
conversation_flant5.predict(text=text_input, query=sentiment_query);
print("\n- Location: ")
conversation_flant5.predict(text=text_input, query=location_query);
print("\n- Time: ")
conversation_flant5.predict(text=text_input, query=time_query);
print("\n\n                          ----====----\n")

***************************
* Claim: claims/claim1.json
***************************
Original content:
-----------------
Subject: Claim for Recent Car Accident - Policy Number: AC-987654321
Content:
Dear Parasol Insurance,

I hope this email finds you well. My name is Sarah Turner, and I am writing to file a claim for a recent car accident that occurred on January 2nd, 2024, at approximately 3:30 PM. My policy number is AC-987654321.

The accident took place at the intersection of Birch Street and Willow Avenue in the city of Evergreen. I was driving my vehicle, a black Toyota Camry with license plate number DEF-456, heading south on Birch Street. At the intersection, the traffic signal was green, and I proceeded through the intersection.

At the same time, another vehicle, a blue Chevrolet Traverse with license plate number GHI-789, was traveling west on Willow Avenue. Unfortunately, the driver failed to stop at the red traffic signal, resulting in a collision with the front passenger 


ご覧のとおり、Flan-T5-Largeは770百万パラメータのモデルであるため、一部の結果を生成するのが速いです。しかし、その結果は精度や詳細性に欠けています。したがって、ある程度は機能しますが、7億パラメータのMistral-7Bから得られる結果には及びません。

LLMを扱う際の技術は、求めるパフォーマンスと精度の間、ならびにそれに伴うリソースやコストとのバランスを見つけることです。

そのため、データが変化したり、モデルが進化したりする際に、常に期待通りの挙動が得られるようにするための信頼性チェックを行うことが重要です。